In [1]:
import sqlite3

# Create SQLite database
conn = sqlite3.connect("cost_optimization.db")
cursor = conn.cursor()

# Create products table
cursor.execute("""
CREATE TABLE products (
    product_id INTEGER PRIMARY KEY,
    product_name TEXT,
    department TEXT
)
""")

# Create costs table
cursor.execute("""
CREATE TABLE costs (
    cost_id INTEGER PRIMARY KEY,
    product_id INTEGER,
    cost INTEGER,
    year INTEGER,
    FOREIGN KEY (product_id) REFERENCES products(product_id)
)
""")

conn.commit()
print("Tables created successfully")


Tables created successfully


In [2]:
# Insert data into products
products_data = [
    (1, 'Server A', 'IT'),
    (2, 'Server B', 'IT'),
    (3, 'Laptop', 'IT'),
    (4, 'Chair', 'HR'),
    (5, 'Table', 'HR'),
    (6, 'Router', 'Networking'),
    (7, 'Switch', 'Networking')
]

cursor.executemany("INSERT INTO products VALUES (?, ?, ?)", products_data)

# Insert data into costs
costs_data = [
    (1, 1, 60000, 2023),
    (2, 2, 55000, 2023),
    (3, 3, 70000, 2023),
    (4, 4, 8000, 2023),
    (5, 5, 12000, 2023),
    (6, 6, 30000, 2023),
    (7, 7, 28000, 2023)
]

cursor.executemany("INSERT INTO costs VALUES (?, ?, ?, ?)", costs_data)

conn.commit()
print("Data inserted successfully")


Data inserted successfully


In [3]:
import pandas as pd

def run_query(query):
    return pd.read_sql_query(query, conn)


In [4]:
query1 = """
SELECT p.department, SUM(c.cost) AS total_cost
FROM products p
JOIN costs c
ON p.product_id = c.product_id
GROUP BY p.department;
"""
run_query(query1)


,department,total_cost
0,HR,20000
1,IT,185000
2,Networking,58000


In [5]:
query2 = """
SELECT p.product_name, AVG(c.cost) AS avg_cost
FROM products p
JOIN costs c
ON p.product_id = c.product_id
GROUP BY p.product_name;
"""
run_query(query2)


,product_name,avg_cost
0,Chair,8000.0
1,Laptop,70000.0
2,Router,30000.0
3,Server A,60000.0
4,Server B,55000.0
5,Switch,28000.0
6,Table,12000.0


In [6]:
query3 = """
SELECT p.product_name, c.cost
FROM products p
JOIN costs c
ON p.product_id = c.product_id
WHERE c.cost > (
    SELECT AVG(cost) FROM costs
);
"""
run_query(query3)


,product_name,cost
0,Server A,60000
1,Server B,55000
2,Laptop,70000


In [7]:
#benchmarking

query4 = """
SELECT p.product_name, p.department, c.cost
FROM products p
JOIN costs c
ON p.product_id = c.product_id
WHERE c.cost > (
    SELECT AVG(c2.cost)
    FROM products p2
    JOIN costs c2
    ON p2.product_id = c2.product_id
    WHERE p2.department = p.department
);
"""
run_query(query4)


,product_name,department,cost
0,Laptop,IT,70000
1,Table,HR,12000
2,Router,Networking,30000


In [8]:
query5 = """
SELECT p.product_name, c.cost
FROM products p
JOIN costs c
ON p.product_id = c.product_id
ORDER BY c.cost DESC
LIMIT 3;
"""
run_query(query5)


,product_name,cost
0,Laptop,70000
1,Server A,60000
2,Server B,55000


SQL analysis revealed that the IT department contributes the highest overall cost. Several products exceed both department-level and global average costs, making them strong candidates for cost optimization through vendor renegotiation, redesign, or alternative sourcing.

In [9]:
queries = """
-- Department-wise Total Cost
SELECT p.department, SUM(c.cost) AS total_cost
FROM products p
JOIN costs c ON p.product_id = c.product_id
GROUP BY p.department;

-- High-Cost Products
SELECT p.product_name, c.cost
FROM products p
JOIN costs c ON p.product_id = c.product_id
WHERE c.cost > (SELECT AVG(cost) FROM costs);
"""

with open("queries.sql", "w") as f:
    f.write(queries)

print("queries.sql created")


queries.sql created


In [14]:
from google.colab import files
files.download("queries.sql")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>